# Importações e tratamentos

Independente de qual seção executar, execute essa primeiro.

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
mpl.rcParams['figure.dpi'] = 300

sns.set_palette("husl")
sns.set_style("whitegrid")

Carregando CSVs e gerando arquivos parquet

In [4]:
raw_dfs: dict[str, pd.DataFrame] = dict()

In [5]:
circuits_df_raw = pd.read_csv(
    ".data/circuits.csv",
    index_col="circuitId"
)
circuits_df_raw["alt"] = pd.to_numeric(circuits_df_raw["alt"], errors="coerce")
circuits_df_raw.to_parquet(".data_parquet/circuits.parquet")
raw_dfs["circuits"] = circuits_df_raw
del circuits_df_raw

In [6]:
constructor_results_df_raw = pd.read_csv(
    ".data/constructor_results.csv",
    index_col="constructorResultsId"
)
constructor_results_df_raw.to_parquet(".data_parquet/constructor_results.parquet")
raw_dfs["constructor_results"] = constructor_results_df_raw
del constructor_results_df_raw

In [7]:
constructor_standings_df_raw = pd.read_csv(
    ".data/constructor_standings.csv",
    index_col="constructorStandingsId"
)
constructor_standings_df_raw.to_parquet(".data_parquet/constructor_standings.parquet")
raw_dfs["constructor_standings"] = constructor_standings_df_raw
del constructor_standings_df_raw

In [8]:
constructors_df_raw = pd.read_csv(
    ".data/constructors.csv",
    index_col="constructorId"
)
constructors_df_raw.to_parquet(".data_parquet/constructors.parquet")
raw_dfs["constructors"] = constructors_df_raw
del constructors_df_raw

In [9]:
driver_standings_df_raw = pd.read_csv(
    ".data/driver_standings.csv",
    index_col="driverStandingsId"
)
driver_standings_df_raw.to_parquet(".data_parquet/driver_standings.parquet")
raw_dfs["driver_standings"] = driver_standings_df_raw
del driver_standings_df_raw

In [10]:
drivers_df_raw = pd.read_csv(
    ".data/drivers.csv",
    index_col="driverId"
)
drivers_df_raw["number"] = pd.to_numeric(drivers_df_raw["number"], errors="coerce")
drivers_df_raw["dob"] = pd.to_datetime(drivers_df_raw["dob"], errors="coerce")
drivers_df_raw.to_parquet(".data_parquet/drivers.parquet")
raw_dfs["drivers"] = drivers_df_raw
del drivers_df_raw

In [11]:
lap_times_df_raw = pd.read_csv(
    ".data/lap_times.csv",
    index_col=["raceId", "driverId", "lap"]
)
# Redundant data
lap_times_df_raw = lap_times_df_raw.drop(columns=["time"])
lap_times_df_raw.to_parquet(".data_parquet/lap_times.parquet")
raw_dfs["lap_times"] = lap_times_df_raw
del lap_times_df_raw

In [19]:
races_df_raw = pd.read_csv(
    ".data/races.csv",
    index_col="raceId"
)
races_df_raw["datetime"] = pd.to_datetime(races_df_raw["date"] + " " + races_df_raw["time"], errors="coerce")
races_df_raw = races_df_raw.drop(columns=["date", "time"])
races_df_raw.to_parquet(".data_parquet/races.parquet")
raw_dfs["races"] = races_df_raw
del races_df_raw

In [52]:
pit_stops_df_raw = pd.read_csv(
    ".data/pit_stops.csv",
    index_col=["raceId", "driverId", "stop"]
)
pit_stops_df_raw = pit_stops_df_raw.drop(columns=["duration"])
pit_stops_df_raw.loc

lap      time  milliseconds
raceId driverId stop                             
841    153      1       1  17:05:23         26898
       30       1       1  17:05:52         25021
       17       1      11  17:20:48         23426
       4        1      12  17:22:34         23251
       13       1      13  17:24:10         23842
...                   ...       ...           ...
1110   4        2      29  15:59:01         23798
       830      2      30  16:00:16         23012
       848      3      33  16:07:06         23529
       858      3      34  16:09:09         23109
       1        3      42  16:23:05         23156

[10089 rows x 3 columns]

# Visualizações

## Construtores

### Tratamento do DataFrame

Ignorando valores não-utilizados.

In [ ]:
constructors_df_raw = pd.read_csv(".data/constructors.csv")
constructors_df_raw

In [ ]:
constructors_df = constructors_df_raw[[
    "constructorId",
    "name",
    "nationality"
]]
constructors_df = constructors_df.set_index("constructorId")
constructors_df

### Distribuição de nacionalidades

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
# SNS count plot order by value counts
sns.countplot(constructors_df, y="nationality", order=constructors_df["nationality"].value_counts().index)
plt.show()

Nota-se:

- Há um domínio de equipes britânicas e americanas no esporte em toda sua história.
  - Isso faz sentido levando em conta o poderio econômico desses países, além do fato da Grã-Bretanha ser até hoje o país que mais consume automobilismo.
- Pelas mesmas razões, outros países europeus menores apresentam uma alta quantidade de construtores também.
- Nota-se que há muitos países que contém apenas uma equipe. Há casos como o Brasil, com equipes como a Copersucar, e casos de "equipes fantasma", que são abordadas na seção a seguir.

### Filtrando construtores "fantasma"

Por questões de curiosidade, ao investigar qual era a equipe da Rodésia do Sul, descobri que se trata de uma construtora que se inscreveu para apenas uma prova na década de 60, mais especificamente, o GP da África do Sul de 1965. Apesar da inscrição ter sido feita, a equipe não chegou nem a participar do Grande Prêmio. (https://www.statsf1.com/en/realpha.aspx)

Para filtrar os construtores apresentados e exigir um pouco mais de consistência dos nossos dados, iremos filtrar todos os construtores que realizaram ao menos 5 voltas em toda sua história. Para isso, precisaremos usar o conjunto de dados de resultados de corridas, filtrando-os por `constructorId`.

In [ ]:
result_df_raw = pd.read_csv(".data/results.csv")
result_df_raw

In [ ]:
if "laps" in constructors_df.columns:
    constructors_df = constructors_df.drop("laps", axis=1)
constructors_df_laps = result_df_raw[[
    "constructorId",
    "laps"
]].groupby("constructorId").sum()
constructors_df = constructors_df.join(constructors_df_laps)
constructors_df = constructors_df.fillna(0).astype({"laps": int})
constructors_df

<text style="color: red">Nota-se: foi usado agrupamento por `constructorId`, ignorando completamente o fato de que dois (ou mais, a depender da temporada) pilotos podem atuar por uma mesma equipe em determinada corrida. Para nossos propósitos de filtragem, isso é irrelevante.</text>

<a id='constructor_lap_distribution'></a>
Quais equipes com mais voltas-piloto na história da Fórmula 1? Para facilitar visualização, iremos limitar às 15 primeiras.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

constructors_df.sort_values("laps", ascending=False).head(15).plot(kind="bar", x="name", y="laps", ax=ax)
ax.set_title("Top 15 Constructors by Laps")
ax.set_xlabel("Constructor")
ax.set_ylabel("Laps")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Novamente, esse resultado faz todo sentido levando em conta que a Scuderia Ferrari foi a única equipe a participar de todas as temporadas da Fórmula 1 desde 1950.

Voltando ao tema principal dessa seção, queremos avaliar a distribuição de nacionalidades excluindo as equipes fantasma.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))
LAPS = 5
filtered_constructors_df = constructors_df.query("laps >= @@LAPS")
ignored_constructors_df = constructors_df.query("laps < @@LAPS")
sns.countplot(
    filtered_constructors_df,
    y="nationality",
    order=filtered_constructors_df["nationality"].value_counts().index,
    ax=ax)
ax.set_title(f"Nationality distribution of Constructors with at least {LAPS} laps")
ax.set_xlabel("Count")
ax.set_ylabel("Nationality")
plt.yticks(rotation=45)
plt.show()

### Equipes fantasma filtradas e o caso da Equipe Lotus

Para efeito de comparação, vamos verificar quantas equipes nunca completaram 20 voltas em toda sua história.

In [ ]:
ignored_constructors_df

Um contexto histórico precisa ser levado em conta nessa análise. No passado, era comum algumas construtoras usar diferentes motores em algumas etapas, então, embora a equipe Lotus tenha tido um número relevante de voltas em toda sua história...

In [ ]:
lotus_df = constructors_df.query("name.str.contains('Lotus')", engine="python")
lotus_df

Essa iteração da equipe (dentre as outras listada acima) não realizou nenhuma volta.

A Equipe Lotus já pertenceu a diferentes grupos, e cada uma desses grupos irão render uma diferente "iteração" da equipe. Infelizmente, não há como fazer uma aglutinação dessas diferentes iterações em um só registro de forma trivial.

Espera-se também que isso tenha impactado o gráfico de distribuição de voltas-piloto por equipe. Para investigar, iremos aglutinar todas as iterações britânicas da equipe Lotus, somar a quantidade de voltas e qual seria o resultado dentre os outros construtores.

In [ ]:
total_laps = lotus_df["laps"].sum()
total_laps

**Esse resultado colocaria a Lotus como a quarta equipe com mais voltas-piloto na história da Fórmula 1, ultrapassando Red Bull, Renault, Sauber e Tyrrell.**

### [Distribuição de voltas](#constructor_lap_distribution)

### Distribuição de vitórias

In [ ]:
if "wins" in constructors_df.columns:
    constructors_df = constructors_df.drop("wins", axis=1)
constructors_df_wins = result_df_raw[[
    "constructorId",
    "position"
]].query("position == '1'").groupby("constructorId").count()
constructors_df_wins.columns = ["wins"]
constructors_df = constructors_df.join(constructors_df_wins)
constructors_df = constructors_df.fillna(0).astype({"wins": int})
constructors_df

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9))

constructors_df.sort_values("wins", ascending=False).head(15).plot(kind="bar", x="name", y="wins", ax=ax)
ax.set_title("Top 15 Constructors by Wins")
ax.set_xlabel("Constructor")
ax.set_ylabel("Wins")
plt.xticks(rotation=45)
plt.tight_layout()
plt.gcf().set_dpi(500)
plt.show()

# Análise de resultados

In [ ]:
result_df_raw[result_df_raw["grid"] == 0]

In [ ]:
status_df_raw = pd.read_csv(".data/status.csv")
status_df_raw

## Distribuição de causas de não-finalização

Podendo ser desqualificação ou abandono.

In [ ]:
dnf_df = result_df_raw[[
    "driverId",
    "raceId",
    "grid",
    "position",
    "statusId",
]].copy()
dnf_df["grid"] = pd.to_numeric(dnf_df["grid"], "coerce")
dnf_df["position"] = pd.to_numeric(dnf_df["position"], "coerce")
dnf_df = dnf_df[dnf_df["position"].isnull()]
dnf_df = dnf_df.join(status_df_raw.set_index("statusId"), on="statusId")
dnf_df.drop(columns=["statusId"], inplace=True)
dnf_df

Histograma das principais causas de não-finalização

In [ ]:
dnf_df["status"].value_counts().head(15).plot(kind="bar")

Ignorando pilotos que não haviam qualificado

In [ ]:
exclude_status = ["Did not qualify", "Did not prequalify", "Not classified"]
dnf_df.query("status not in @@exclude_status", engine="python")["status"].value_counts().head(15).plot(kind="bar")

Também é possível separar por piloto (p.e. abandonos do Lewis Hamilton)

In [ ]:
dnf_df.query("driverId == 1 & status not in @@exclude_status", engine="python")["status"].value_counts().head(15).plot(kind="bar")

Será que existe diferentes distribuições de abandono para cada posição de largada?

In [ ]:
# It would be interesting to show how many DNFs we have in this. For now, only ignoring.
import numpy as np
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots()
x = np.unique(dnf_df['grid'].sort_values().values)

def update(frame):
    this_x = x[frame]
    subset = dnf_df.query("grid == @@this_x & status not in @@exclude_status", engine="python")["status"].value_counts()
    this_data = subset.head(15)
    total = subset.sum()
    percentage = (this_data / total) * 100
    ax.cla()
    percentage.plot(kind="bar", ax=ax)
    if this_x == 0:
        title = "Histogram for starting from the pitlane"
    else:
        title = f"Histogram for starting grid at {this_x}"
    ax.set_title(title)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(PercentFormatter())

anim = animation.FuncAnimation(fig=fig, func=update, frames=len(x), interval=1000)
anim.save(".animations/dnfs.mp4")

## Para quem concluiu: distribuição de resultados por grid x posição

Ignorando resultados obtidos a partir de largada do pitlane

In [ ]:
position_grid_df = result_df_raw[[
    "grid",
    "position",
]].copy()
position_grid_df["grid"] = pd.to_numeric(position_grid_df["grid"], "coerce")
position_grid_df["position"] = pd.to_numeric(position_grid_df["position"], "coerce")
position_grid_df = position_grid_df.dropna()
position_grid_df["position"] = position_grid_df["position"].astype(int)
ignored_pitlane_df = position_grid_df.query("grid > 0")
plt.hist2d(ignored_pitlane_df["grid"], ignored_pitlane_df["position"], (10, 10))
plt.colorbar()
plt.show()

Histograma em uma dimensão, um frame para cada posição de largada. Incluindo largadas do pitlane.

In [ ]:
import numpy as np
import matplotlib.animation as animation
from matplotlib.ticker import MaxNLocator, PercentFormatter

fig, ax = plt.subplots()
x = np.unique(position_grid_df['grid'].sort_values().values)

def update(frame):
    if frame < 10:
        this_x = 0
    else:
        this_x = x[frame - 10]
    this_data = position_grid_df[position_grid_df["grid"] == this_x]["position"].values
    ax.cla()
    sns.histplot(this_data, bins=33, binrange=(1, 33), ax=ax, stat='percent')
    if this_x == 0:
        title = "Histogram for starting from the pitlane"
    else:
        title = f"Histogram for starting grid at {this_x}"
    ax.set_title(title)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(PercentFormatter())
anim = animation.FuncAnimation(fig=fig, func=update, frames=len(x) + 10, interval=200)
anim.save(".animations/gridxpos.mp4")

In [ ]:
fig, ax = plt.subplots()
x = np.unique(ignored_pitlane_df['grid'].sort_values().values)

def update(frame):
    this_x = x[frame]
    this_data = ignored_pitlane_df[ignored_pitlane_df["grid"] == this_x]["position"]
    this_data = (-this_data) + this_x
    ax.cla()
    this_data.value_counts().sort_index(ascending=False).plot(kind="bar", ax=ax)
    if this_x == 0:
        title = "Histogram for starting from the pitlane"
    else:
        title = f"Histogram for starting grid at {this_x}"
    ax.set_title(title)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax.yaxis.set_major_formatter(PercentFormatter())
anim = animation.FuncAnimation(fig=fig, func=update, frames=len(x), interval=200)
anim.save(".animations/gridxpos_test.mp4")